# Notebook: Einführung in Gradio

Dieses Notebook behandelt die wichtigsten Grundlagen von Gradio.

Offizielle Dokumentation: [Gradio Docs](https://gradio.app/docs/)

In [48]:
%%capture 
# Installieren wir zunächst Gradio
!pip install gradio

In [49]:
import gradio as gr

### Beispiel 1: Einfaches User Interface (Beispiel aus den Folien)

In Gradio können wir mit wenigen Zeilen Code eine Benutzeroberfläche (UI) für unsere LLM Anwendungen erstellen.
Anders als bei Web-Frameworks wie Flask oder FastAPI, müssen wir uns nicht um die Details der Webentwicklung kümmern, sondern können uns auf die Funktionalität konzentrieren.

Die einfachste Möglichkeit, eine UI zu erstellen, ist die Verwendung der `gr.Interface` Klasse.
`gr.Interface` benötigt mindestens drei Argumente:


```python

- `fn`:  Die Funktion, die aufgerufen wird, wenn der Benutzer die Eingaben sendet.
  Die Funktion, um die eine Benutzeroberfläche (UI) gebaut wird.

- `inputs`:  
  Die Gradio-Komponente(n), die als Eingabe verwendet werden.  
  👉 Die Anzahl der Komponenten muss der Anzahl der Argumente deiner Funktion entsprechen.  

- `outputs`:  
  Die Gradio-Komponente(n), die als Ausgabe verwendet werden.  
  👉 Die Anzahl der Komponenten muss der Anzahl der Rückgabewerte deiner Funktion entsprechen.  


In [50]:
def greet(name, intensity):
    return "Hello, " + name + "!" * int(intensity)

demo = gr.Interface(
    fn=greet,
    inputs=["text", "slider"],
    outputs=["text"],
)

# Mit share=False wird kein Link generiert, um die App zu öffentlich (!) zu teilen. 
# Hinweis: Standardmäßig ist share=False, was bedeutet, dass kein Link generiert wird, um die App öffentlich zu teilen.
demo.launch(share=False)

* Running on local URL:  http://127.0.0.1:7881
* To create a public link, set `share=True` in `launch()`.


### Beispiel 2: User Interface mit verschiedenen Komponenten

Eine Dokumentation aller Komponenten findest du hier: [Gradio Components](https://www.gradio.app/docs/gradio)

```python

**Funktion `sentence_builder`:**
Generiert einen englischen Satz aus verschiedenen Eingaben mittels f-String und `join()`.

**Interface-Komponenten:**

- **Slider**: Zahlenbereich (2-20) mit Standardwert
- **Dropdown**: Einfachauswahl für Tiere
- **CheckboxGroup**: Mehrfachauswahl für Länder
- **Radio**: Einzelauswahl für Orte
- **Dropdown (multiselect)**: Mehrfachauswahl für Aktivitäten mit Voreinstellung
- **Checkbox**: Ja/Nein für Tageszeit

**Key Features**

- `info`: Zusätzliche Beschreibung unter dem Label
- `value`: Standardwerte setzen
- `multiselect=True`: Mehrfachauswahl ermöglichen
- `examples`: Vorgefertigte Testdaten für schnelle Demo

**Output**

Beispiel: "The 4 cats from Japan and Pakistan went to the park where they ate and swam until the morning"


In [51]:
# Die Reihenfolge der Parameter in der Funktion muss mit der Reihenfolge der Inputs übereinstimmen
def sentence_builder(quantity, animal, countries, place, activity_list, morning):
    return f"""The {quantity} {animal}s from {" and ".join(countries)} went to the {place} where they {" and ".join(activity_list)} until the {"morning" if morning else "night"}"""


demo = gr.Interface(
    fn=sentence_builder,
    inputs=[
        # Die Info wird unterhalb von Label dargestellt
        gr.Slider(2, 20, value=4, label="Count",
                  info="Choose between 2 and 20"),
        gr.Dropdown(["cat", "dog", "bird"], label="Animal",
                    info="Will add more animals later!"),
        gr.CheckboxGroup(["USA", "Japan", "Pakistan"],
                         label="Countries", info="Where are they from?"),
        gr.Radio(["park", "zoo", "road"], label="Location",
                 info="Where did they go?"),
        gr.Dropdown(["ran", "swam", "ate", "slept"],
                    # Voreinstellung für die Dropdown-Auswahl
                    value=["swam", "slept"],
                    multiselect=True,
                    label="Activity",
                    info="Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed auctor, nisl eget ultricies aliquam, nunc nisl aliquet nunc, eget aliquam nisl nunc vel nisl."
                    ),
        gr.Checkbox(label="Morning", info="Did they do it in the morning?"),
    ],
    outputs=gr.Textbox(
        label="Output", info="This is the generated sentence based on your inputs."),  # Ausgabeformat
    examples=[
        # Mithilfe von Examples können wir vorgefertigte Eingaben bereitstellen, die der Nutzer auswählen kann um die App zu testen
        [2, "cat", ["Japan", "Pakistan"], "park", ["ate", "swam"], True],
        [4, "dog", ["Japan"], "zoo", ["ate", "swam"], False],
        [10, "bird", ["USA", "Pakistan"], "road", ["ran"], False],
        [8, "cat", ["Pakistan"], "zoo", ["ate"], True],
    ]
)

demo.launch()

* Running on local URL:  http://127.0.0.1:7882
* To create a public link, set `share=True` in `launch()`.


### Beispiel 3: Blocks

1. `Blocks`

- `gr.Blocks` ist das Container-Layout-System in Gradio, mit dem komplexe Benutzeroberflächen modular aufgebaut werden können.  
- Innerhalb eines Blocks lassen sich verschiedene Layout-Elemente wie `Row`, `Column` oder Tabs definieren.  
- Man kann auch Themes (z. B. `gr.themes.Citrus()`) zuweisen, um das Design global zu steuern.  
- `with` in Python (Kontextmanager): wird als als **Kontextmanager** verwendet, um einen bestimmten Gültigkeitsbereich für Objekte zu eröffnen. In Gradio bedeutet das: Alle Komponenten, die innerhalb von `with gr.Row():` oder `with gr.Column():` geschrieben werden, gehören automatisch zu diesem Layout. Dadurch spart man sich explizite Zuordnungen und schreibt übersichtlicheren Code, da die Hierarchie durch Einrückung klar erkennbar ist.  


---

2. `Row`

- `gr.Row()` ordnet die enthaltenen Komponenten **nebeneinander in einer horizontalen Reihe** an.  
- Alle Elemente innerhalb einer Row teilen sich den verfügbaren Platz gleichmäßig (sofern keine speziellen Breiten angegeben sind).  
- Typisches Beispiel: Mehrere Eingabefelder wie Textboxen oder Zahlenfelder, die nebenläufig erscheinen sollen.  

---

3. `Column`

- `gr.Column()` ordnet die enthaltenen Komponenten **untereinander in einer vertikalen Spalte** an.  
- Sie eignet sich, um Inhalte strukturiert übereinander darzustellen, z. B. einen Button und das dazugehörige Ausgabefeld.  
- Man kann `Row` und `Column` auch verschachteln, um flexible Layouts zu gestalten (z. B. eine Zeile mit Spalten, die wiederum Reihen enthalten).  


In [52]:
import gradio as gr

# Funktion, die ausgeführt wird, wenn der Button geklickt wird
def begruessung(vorname, nachname, alter):
    return f"Hallo, {vorname} {nachname}! Du bist {alter} Jahre alt."

# Erstellen des Blocks
with gr.Blocks(theme=gr.themes.Citrus()) as demo:
    # Vertikales Layout
    with gr.Row():
        vorname_input = gr.Textbox(label="Vorname")
        nachname_input = gr.Textbox(label="Nachname")
        alter_input = gr.Number(label="Alter")
    with gr.Column():
        begruessung_button = gr.Button("Sag Hallo")
        output_text = gr.Textbox(label="Ausgabe")

    # Event: Button klickt -> Funktion ausführen -> Ergebnis in output_text schreiben
    begruessung_button.click(begruessung, 
                             inputs=[vorname_input, 
                                     nachname_input, 
                                     alter_input], 
                             outputs=output_text)

demo.launch()

* Running on local URL:  http://127.0.0.1:7883
* To create a public link, set `share=True` in `launch()`.


### Beispiel 4: Events

#### A) Click Event

👉 Erklärung: Durch `.click()` wird eine Funktion `fn` aufgerufen. `.click()` wird von `Button`, `ClearButton` und `UploadButton` unterstützt. 

In [53]:
import gradio as gr

# Funktion, die aufgerufen wird, wenn der Button geklickt wird
def greet(name):
    return f"Hallo {name}!"

with gr.Blocks() as demo:
    # Eingabefeld für den Namen
    name = gr.Textbox(label="Name eingeben")
    
    # Ausgabefeld für die Antwort
    output = gr.Textbox(label="Antwort")
    
    # Button, der das Event auslöst
    btn = gr.Button("Grüßen")
    
    # Event: Wenn Button geklickt wird (.click),
    # dann wird die Funktion greet() ausgeführt.
    # Inputs = "name", Outputs = "output"
    btn.click(fn=greet, inputs=name, outputs=output)

# demo.launch()

#### B) Submit Event

👉 Erklärung: Durch `.submit()` wird eine Funktion `fn` aufgerufen, sobald der Nutzer Enter drückt oder ein Formular absendet. `.submit()` wird von `Textbox` und `Chatbot` unterstützt.

In [54]:
import gradio as gr

# Funktion: gibt den Text rückwärts zurück
def reverse_text(text):
    return text[::-1]

with gr.Blocks() as demo:
    # Eingabefeld: Nutzer tippt Text und drückt Enter
    inp = gr.Textbox(label="Text eingeben und Enter drücken")
    
    # Ausgabe: zeigt den umgedrehten Text
    out = gr.Textbox(label="Umgekehrter Text")
    
    # Event: Wenn der Nutzer Enter drückt (.submit),
    # wird die Funktion reverse_text() ausgeführt
    inp.submit(fn=reverse_text, inputs=inp, outputs=out)

# demo.launch()

#### C) Change Event

👉 Erklärung: Durch `.change()` wird eine Funktion `fn` aufgerufen, sobald sich der Wert eines Elements ändert. `.change()` wird von `Dropdown`, `Checkbox`, `CheckboxGroup`, `Radio`, `Slider`, `Number`, `ColorPicker`, `Image` und `File` unterstützt.

In [55]:
import gradio as gr

# Funktion: beschreibt die gewählte Farbe
def describe_color(color):
    return f"Du hast {color} ausgewählt."

with gr.Blocks() as demo:
    # Dropdown mit drei Auswahlmöglichkeiten
    dropdown = gr.Dropdown(choices=["Rot", "Grün", "Blau"], label="Farbe wählen")
    
    # Ausgabe: zeigt die Beschreibung
    out = gr.Textbox(label="Beschreibung")
    
    # Event: Wenn die Auswahl im Dropdown geändert wird (.change),
    # wird die Funktion describe_color() ausgeführt
    dropdown.change(fn=describe_color, inputs=dropdown, outputs=out)

# demo.launch()

#### D) Input Event

👉 Erklärung: Durch `.input()` wird eine Funktion `fn` während der Eingabe im Element aufgerufen (z. B. bei jedem Tastendruck). `.input()` wird von `Textbox`, `Number` und `Slider` unterstützt.

In [56]:
import gradio as gr

# Funktion: berechnet die Länge des eingegebenen Textes
def live_length(text):
    return f"Länge: {len(text)} Zeichen"

with gr.Blocks() as demo:
    # Eingabefeld, wo man live tippt
    inp = gr.Textbox(label="Gib etwas ein...")
    
    # Ausgabe: zeigt sofort die Länge des Textes
    out = gr.Textbox(label="Länge")
    
    # Event: Während der Nutzer tippt (.input),
    # wird die Funktion live_length() sofort aufgerufen
    inp.input(fn=live_length, inputs=inp, outputs=out)

# demo.launch()

#### E) Verkettung mit `.then()

👉 Erklärung: Durch `.then()` wird eine Funktion `fn` aufgerufen, nachdem eine andere Funktion (z. B. durch `.click()` oder `.submit()`) erfolgreich ausgeführt wurde. `.then()` wird von allen Elementen unterstützt, die zuvor ein Event wie `.click()`, `.submit()` oder `.input()` auslösen.
`

In [57]:
import gradio as gr

# Funktion 1: Begrüßt den Nutzer
def greet(name):
    return f"Hallo {name}!"

# Funktion 2: Gibt die Länge des Namens zurück
def name_length(greeting):
    return f"Länge der Begrüßung: {len(greeting)} Zeichen"

with gr.Blocks() as demo:
    name = gr.Textbox(label="Name eingeben")
    output1 = gr.Textbox(label="Begrüßung")
    output2 = gr.Textbox(label="Länge der Begrüßung")
    btn = gr.Button("Grüßen")
    
    # Event: zuerst greet() aufrufen, dann name_length() mit .then()
    btn.click(fn=greet, inputs=name, outputs=output1).then(fn=name_length, inputs=output1, outputs=output2)

# demo.launch()

### Beispiel 5: Verschalten von Layouts und Scaling

In [58]:
import gradio as gr

with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column(scale=1):
            gr.Textbox(label="Links 1")
            gr.Textbox(label="Links 2")
        with gr.Column(scale=2):
            gr.Textbox(label="Rechts 1")
            gr.Textbox(label="Rechts 2")

# demo.launch()

### Beispiel 6: Tabs

Tab() ist ein Layoutelement. Innerhalb des Tabs definierte Komponenten sind sichtbar, wenn dieser Tab ausgewählt ist.

In [59]:
with gr.Blocks() as demo:
    with gr.Tab("Tab 1"):
        gr.Textbox(label="Text 1")
        gr.Textbox(label="Text 2")
    with gr.Tab("Tab 2"):
        gr.Textbox(label="Text 1")
        gr.Textbox(label="Text 2")
        
# demo.launch()

### Beispiel 7: Chatbot

In [60]:
history = [
    {"role": "user", "content": "What time is it?"},
    {"role": "assistant", "content": "It's 3 PM."},
]

with gr.Blocks() as demo:
    gr.Chatbot(history, type="messages")

# demo.launch()

### Beispiel 8: Chatbot mit Eingabefeld und Button

In [66]:
LLM_URL = "http://132.199.138.16:11434/v1"
LLM_MODEL = "gemma3:12b" 

In [67]:
import openai

client = openai.OpenAI(
    base_url=LLM_URL,
    api_key="ollama",
)

history = [
    {"role": "user", "content": "What time is it?"},
    {"role": "assistant", "content": "It's 3 PM."},
]


def add_user_message(message, chat_history):
    """Fügt sofort die User-Message hinzu"""
    if message.strip():
        chat_history.append({"role": "user", "content": message})
    return chat_history, ""


def get_llm_response(chat_history):
    """Holt die LLM-Antwort und fügt sie hinzu"""
    if not chat_history or chat_history[-1]["role"] != "user":
        return chat_history

    # API-Request an das LLM using OpenAI client
    response = client.chat.completions.create(
        model=LLM_MODEL,
        messages=chat_history,
        stream=False
    )

    assistant_reply = response.choices[0].message.content
    chat_history.append({"role": "assistant", "content": assistant_reply})

    return chat_history


with gr.Blocks() as demo:
    chatbot = gr.Chatbot(value=history, type="messages")
    with gr.Row():
        with gr.Column(scale=1):
            user_input = gr.Textbox(
                label="User Input", placeholder="Type a message...")
        with gr.Column(scale=0.2):
            send_btn = gr.Button("Send", variant="primary")

    # Chain die Funktionen: erst User-Message hinzufügen, dann LLM-Antwort holen
    # .then() in Gradio verkettet zwei Funktionen, sodass die zweite automatisch ausgeführt wird, nachdem die erste abgeschlossen ist.
    # Die Outputs der ersten Funktion werden als Inputs an die zweite Funktion weitergegeben, wodurch eine sequenzielle Abarbeitung entsteht.
    send_btn.click(
        fn=add_user_message,
        inputs=[user_input, chatbot],
        outputs=[chatbot, user_input]
    ).then(
        fn=get_llm_response,
        inputs=[chatbot],
        outputs=[chatbot]
    )

demo.launch()

/opt/homebrew/Caskroom/miniforge/base/envs/env_nils_hellwig/lib/python3.12/site-packages/gradio/layouts/column.py:59: UserWarning: 'scale' value should be an integer. Using 0.2 will cause issues.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7886
* To create a public link, set `share=True` in `launch()`.


## Aufgabe 1: Gradio-Anwendung zur Bild-Fragebeantwortung


In dieser Aufgabe implementierst du eine kleine **Aspect-Based Sentiment Analysis (ABSA)** Anwendung mit Gradio.

Ziel ist, dass der Nutzer im UI einen Text und eine Liste von Kategorien eingibt. Die Anwendung soll dann für jede Kategorie das Sentiment (😊 positiv, 😐 neutral, 😞 negativ) im Text erkennen und in einer Tabelle im UI anzeigen.

---

### Schritte

1. **Eingaben prüfen**  
   - Falls kein Text: Fehlermeldung zurückgeben.  
   - Falls keine Kategorie(n): Fehlermeldung zurückgeben.  
   - Hinweis: Kategorien sind **kommagetrennt**.

2. **Prompt erstellen & Modell aufrufen**  
   - Nutze die Funktion `get_user_prompt(text, categories_list)`.  
   - Rufe das LLM mit `llm_text_to_json(user_prompt)` auf.  
   - Ergebnis ist ein JSON mit `items: [{category, sentiment}]`.

3. **Ausgabe formatieren**  
   - Erstelle eine Tabelle mit Spalten: **Kategorie**, **Sentiment**.  
   - Emojis für Sentiment nutzen (optional):
     - 😊 Positiv
     - 😐 Neutral
     - 😞 Negativ  
   - Falls keine Kategorie erkannt → Hinweis ausgeben.

4. **UI bauen**  
   - Nutze Gradio `Blocks` mit:
     - Textbox für den Text  
     - Textbox für Kategorien  
     - Button „Sentiment analysieren“  
     - Tabelle mit Ergebnissen  
     - HTML für Fehlermeldungen  
     - Beispiele (`gr.Examples`) zum Testen

---

## 💡 Hilfen
- Verwende `strip()` und `split(",")`, um Kategorien zu bereinigen.  
- Nutze `if not ...:` für Validierungen.  
- Rückgabewert von `analyze_sentiment_guided`:  
  ```python
  return table_data, error_message
  ```
- Denke an den leeren Array-Fall: Keine Kategorie erkannt → Hinweis anzeigen.



Folgender Code ist gegeben:

In [68]:
from pydantic import BaseModel
from enum import Enum
import json


class Sentiment(str, Enum):
    positive = "positive"
    neutral = "neutral"
    negative = "negative"


class ABSAItem(BaseModel):
    category: str
    sentiment: Sentiment


class ABSAResponse(BaseModel):
    items: list[ABSAItem]


def get_user_prompt(text, categories_list):
    user_prompt = (
        f"Text: {text}\n"
        f"Kategorien: {categories_list}\n"
        "Gib ein JSON-Objekt mit einem 'items' Array zurück, das Objekte mit {category, sentiment} für jede Kategorie enthält. "
        "Das Sentiment soll eines von folgenden sein: positive, neutral, negative. "
        "WICHTIG: Gib nur Kategorien zurück, zu denen im Text tatsächlich ein Sentiment geäußert wird. "
        "Falls keine Stimmung gegenüber den genannten Kategorien im Text ausgedrückt wird, gib ein leeres Array zurück. "
        "Kategorien ohne erkennbares Sentiment sollen NICHT in der Antwort enthalten sein."
    )
    return user_prompt


def llm_text_to_json(user_prompt):
    completion = client.chat.completions.parse(
        temperature=0,
        model=LLM_MODEL,
        messages=[
            {"role": "user", "content": user_prompt}
        ],
        response_format=ABSAResponse,
    )

    output = completion.choices[0].message.content

    structured_output = json.loads(output)
    return structured_output


# === Beispiele für die UI ===
examples = [
    [
        "Das Essen war lecker, aber der Service war sehr langsam.",
        "Essen, Service",
    ],
    [
        "Das Handy hat eine großartige Kamera, aber der Akku ist schwach.",
        "Kamera, Akku, Design",
    ],
    [
        "Die Lieferung kam pünktlich, aber die Verpackung war beschädigt.",
        "Lieferung, Verpackung",
    ],
]

In [69]:
# Hier kannst du die Gradio-App erstellen...

<details>
<summary><b>Lösung anzeigen</b></summary>

```python
def analyze_sentiment_guided(text, categories):
    # Zunächst prüfen, ob die Eingaben gültig sind
    # und entsprechende Fehlermeldungen zurückgeben, wenn nicht.
    if not text.strip():
        return [], "<h1>⚠️ Fehler, Bitte einen Text eingeben.</h1>"
    if not categories.strip():
        return [], "<h1>⚠️ Fehler, Bitte mindestens eine Kategorie eingeben.</h1>"

    categories_list = [c.strip() for c in categories.split(",") if c.strip()]
    if not categories_list:
        return [], "<h1>⚠️ Fehler, Bitte mindestens eine Kategorie eingeben.</h1>"


    # Erstelle den User-Prompt und rufe das LLM auf, um die Sentiment-Analyse durchzuführen
    user_prompt = get_user_prompt(text, categories_list)
    structured_output = llm_text_to_json(user_prompt)


    # Erstelle Tabellen-Daten
    table_data = []
    
    for item in structured_output["items"]:
        if item["sentiment"] == "positive":
            sentiment_text = "😊 Positiv"
        elif item["sentiment"] == "negative":
            sentiment_text = "😞 Negativ"
        else:
            sentiment_text = "😐 Neutral"
        
        table_data.append([item["category"], sentiment_text])

    if not table_data:
        return [], "<h1>ℹ️ Hinweis: Keine Kategorie im Text erwähnt.</h1>"

    return table_data, ""

# Update the Gradio interface to use Table output
with gr.Blocks() as demo:
    gr.Markdown("## Aspect-Based Sentiment Analysis (ABSA) mit Guided JSON")
    
    error_text = gr.HTML(
        label="Status",
        visible=True
    )

    with gr.Row():
        text_input = gr.Textbox(
            label="Text eingeben", placeholder="Gib einen Text ein...", lines=5
        )

    with gr.Row():
        category_list = gr.Textbox(
            label="Kategorien (kommagetrennt)", placeholder="z.B. Preis, Qualität, Lieferung"
        )

    analyze_btn = gr.Button("Sentiment analysieren")
    output_table = gr.Dataframe(
        headers=["Kategorie", "Sentiment"],
        datatype=["str", "str"],
        label="ABSA Ergebnis",
    )

    analyze_btn.click(
        fn=analyze_sentiment_guided,
        inputs=[text_input, category_list],
        outputs=[output_table, error_text],
    )

    gr.Examples(
        examples=examples,
        inputs=[text_input, category_list],
        label="Beispiele",
    )

demo.launch()
```

</details>